In [2]:
import qmps
import xmps
import cirq
from xmps import iMPS
import numpy as np
def ap(A, _abs = True):
    if _abs:
        print(np.around(np.abs(A), 2))
    else:
        print(np.around(A, 2))

In [23]:
from numpy.linalg import svd

def S(ρ):
    # S = Σᵢρᵢlog(ρᵢ) where ρᵢ = |λᵢ|^2 and λᵢ = singular values 
    u, s, v = svd(ρ) 
    s = np.square(np.abs(s)) 
    return np.dot(np.log2(s), s)
    

### Alternating Hamiltonians to get 3 state evolution (didn't work)

In [66]:
from qmps.represent import FullStateTensor, FullEnvironment, Environment, ShallowFullStateTensor, Tensor
from qmps.tools import unitary_to_tensor, environment_from_unitary, tensor_to_unitary, get_env_exact
from xmps.spin import U4
from xmps.iMPS import Map, iMPS
from qmps.time_evolve_tools import merge, put_env_on_left_site, put_env_on_right_site
from tqdm.notebook import tqdm
from scipy.linalg import expm
from scipy.optimize import minimize
import numpy as np
import cirq

def two_site_sim(params, U1, U2,  W, A_input1, A_input2):
    
    """
    Simulate a quantum state of the form:
    
    |    |    |    |    |----|    |
    |    |    |    |-U1-|    |    |
    |    |    |-U2-|    |    |    |
    |    |-U1-|    |    |    |    |
    |-L--|    |----W----|    |--R-|
    |    |U1'-|    |    |    |    |
    |    |    |U2'-|    |    |    |
    |    |    |    |U1'-|    |    |
    |    |    |    |    |----|    |
    
    and return the overlap of these wo states to find parameterisations of U1' and U2' that minimize this overlap.
    
    Inputs: 
    1) params: 2 * N where N is the number of parameters used to specify the unitary U1(2)
    2) U1/U2: A parameterized unitary that decomposes to native cirq gates
    3) W: exponentiated hamiltonian 
    
    Ouputs:
    1) Final state |ψ>
    """
    
    U1_ = ShallowFullStateTensor(2, params[:15])
    U2_ = ShallowFullStateTensor(2, params[15:])
    
    A1_ = unitary_to_tensor(cirq.unitary(U1_))
    A2_ = unitary_to_tensor(cirq.unitary(U2_))

    if A_input1 is None:
        A1 = unitary_to_tensor(cirq.unitary(U1))
        A2 = unitary_to_tensor(cirq.unitary(U2))
    else:
        A1 = A_input1
        A2 = A_input2
    
    A12 = merge(A1, A2)
    A12_ = merge(A1_, A2_)
    
    _, r = Map(A12,A12_).right_fixed_point()
    
    R = Environment(put_env_on_left_site(r), 'θR')
    left = put_env_on_right_site(r.conj().T)
    L = Environment(left, 'θL')

    q = cirq.LineQubit.range(7)

    circuit = cirq.Circuit.from_ops([
        cirq.H(q[4]),
        cirq.CNOT(q[4], q[5]),
        U1(*q[3:5]),
        U2(*q[2:4]),
        U1(*q[1:3]),
        L(*q[0:2]),
        R(*q[5:7]),
        W(*q[2:5]),
        cirq.inverse(U1_)(*q[1:3]),
        cirq.inverse(U2_)(*q[2:4]),
        cirq.inverse(U1_)(*q[3:5]),
        cirq.CNOT(q[4], q[5]),
        cirq.H(q[4])
        ])

    sim = cirq.Simulator()
    X =sim.simulate(circuit).final_state
    return X 

def scar_obj_fun(params, U1, U2,  W, A_input1, A_input2):
    return np.abs(two_site_sim(params, U1, U2,  W, A_input1, A_input2)[0])*2    


params = np.random.randn(30)

class Neel2(cirq.Gate):
    """
    Gate for the very first step of the many body scar algorithm:
    |-𝕀-|-X-|
    """
    
    def num_qubits(self):
        return 2
    
    def _decompose_(self, qubits):
        return [cirq.I(qubits[0]), cirq.X(qubits[1])]
    
    def _circuit_diagram_info_(self, args):
        return 'U1'
    
class Neel1(cirq.Gate):
    """
    Gate for the very first step of the many body scar algorithm:
    |-𝕀-|-𝕀-|
    """

    def num_qubits(self):
        return 2
    
    def _decompose_(self, qubits):
        return [cirq.I(qubits[0]), cirq.I(qubits[1])]
    
    def _circuit_diagram_info_(self, args):
        return 'U2'

def make_random_As(noise = False):
    A = np.random.randn(2,2,2)
    
    if noise:
        ζ = 1e-6*np.random.randn(7)
        a1 = [[1,ζ[0]], [ζ[1],ζ[2]]]
        a2 = [[ζ[4],ζ[5]],[ζ[6],ζ[3]]]
    
    if not noise:
        ζ = 1e-7
        a1 = [[1,0], [0,0]]
        a2 = [[0,ζ],[0,0]]

    A1 = A.copy()
    A2 = A.copy()

    A1[0,:,:] = a1
    A1[1,:,:] = a2

    A2[0,:,:] = a2
    A2[1,:,:] = a1
    
    return [A1, A2]

U1 = Neel1()
U2 = Neel2()

As1, As2 = make_random_As()

TimeSteps = 500
dt = 0.01
initial_params = np.random.randn(30)

P = np.array([[1,0],[0,0]])
X = np.array([[0,1],[1,0]])
H = np.kron(np.kron(P,X),P)
W = Tensor(expm(-1j * H * dt), "W")

final_parameters = []

for i in tqdm(range(TimeSteps)):
    if i == 0:
        res = minimize(scar_obj_fun, initial_params, args = (U1, U2, W, As1, As2), method = "Nelder-Mead", options = {'disp':True})
        U1 = ShallowFullStateTensor(2, res.x[:15])
        U2 = ShallowFullStateTensor(2, res.x[15:])
        final_parameters.append(res.x)
    
    if (i > 0) & (i%2 == 0):
        res = minimize(scar_obj_fun, res.x, args = (U1, U2, W, None, None), method = "Nelder-Mead",options = {'disp':True})
        U1 = ShallowFullStateTensor(2, res.x[:15])
        U2 = ShallowFullStateTensor(2, res.x[15:])
        final_parameters.append(res.x)

    if (i > 0) & (i%2 == 1):
        # replace the first 15 params with the last 15 and vice versa, so the front 15 params now
        # refer to U2, and the last 15 for U1
        flipped_params = res.x.copy()
        flipped_params[:15] = res.x[15:]
        flipped_params[15:] = res.x[:15]
        res = minimize(scar_obj_fun, flipped_params, args = (U2, U1, W, None, None), method = "Nelder-Mead", options = {'disp':True})
        # Flip the parameters back so the refer to the original direction for the subsequent time step
        temp = res.x.copy()
        res.x[:15] = temp[15:]
        res.x[15:] = temp[:15]
        U1 = ShallowFullStateTensor(2, res.x[:15])
        U2 = ShallowFullStateTensor(2, res.x[15:])
        final_parameters.append(res.x)

np.save('many_body_scar_params', np.array(final_parameters))

        
# -----------------------------------------

# """
# What needs to be done:

# 1) make a multi-site state out of 2 separately parameterized unitary matrices -Done

# 2) figure out a ay to extract an environment from this state  -Done
#         - tensor to unitart,
#         - merge,
#         - get_left/right env
        
# 3) Calculate environment    
    
# 4) get entanglement entropy:
#         - reduced density matrix of the environment 

# 5) Poincare map (have no idea what this is)

# """

Optimization terminated successfully.
         Current function value: 0.000005
         Iterations: 721
         Function evaluations: 1123
Optimization terminated successfully.
         Current function value: 0.000006
         Iterations: 941
         Function evaluations: 1406
Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 750
         Function evaluations: 1142
Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 917
         Function evaluations: 1390
Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 1105
         Function evaluations: 1659
Optimization terminated successfully.
         Current function value: 0.000003
         Iterations: 687
         Function evaluations: 1108
Optimization terminated successfully.
         Current function value: 0.000011
         Iterations: 701
         Function evaluations: 1114
Optimization

         Current function value: 0.000187
         Iterations: 603
         Function evaluations: 1239
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 591
         Function evaluations: 1089
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 843
         Function evaluations: 1372
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 580
         Function evaluations: 1064
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 864
         Function evaluations: 1408
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 684
         Function evaluations: 1183
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 690
         Function evaluations: 1194
Optimization terminated successfully.
         Curr

         Current function value: 0.000000
         Iterations: 524
         Function evaluations: 1054
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 649
         Function evaluations: 1222
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 547
         Function evaluations: 1106
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 661
         Function evaluations: 1375
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 633
         Function evaluations: 1212
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 628
         Function evaluations: 1165
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 464
         Function evaluations: 1014
Optimization terminated successfully.
         Curr

         Current function value: 0.000000
         Iterations: 587
         Function evaluations: 1175
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1216
         Function evaluations: 1969
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1256
         Function evaluations: 1977
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 802
         Function evaluations: 1453
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 572
         Function evaluations: 1142
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2012
         Function evaluations: 2897
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 700
         Function evaluations: 1338
Optimization terminated successfully.
         C

         Current function value: 0.000000
         Iterations: 548
         Function evaluations: 1125
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 617
         Function evaluations: 1248
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 583
         Function evaluations: 1191
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1526
         Function evaluations: 2363
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 597
         Function evaluations: 1189
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 484
         Function evaluations: 1045
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 562
         Function evaluations: 1143
Optimization terminated successfully.
         Cur

In [67]:
# have parameters that define a U1 and U2.
# build U1 & U2 - ShallowFullStateTensor
# Turn into  As: unitary_to_tensor(cirq.unitary(U))
# Merge A1 and A2
# get_env_exact on merged A12
# Look at the values of the environment to get the entanglement spectrum
params = np.load('many_body_scar_params.npy')
envs = []
for p in params:
    p1 = p[:15]
    p2 = p[15:]
    U1 = ShallowFullStateTensor(2, p1)
    U2 = ShallowFullStateTensor(2, p2)
    
    A1 = unitary_to_tensor(cirq.unitary(U1))
    A2 = unitary_to_tensor(cirq.unitary(U2))
    
    A12 = merge(A1,A2)
    env = get_env_exact(A12)
    
    envs.append(env)


In [68]:
from numpy.linalg import svd
def env_to_entropy(env):
    λ = env[:,0].reshape(2,2)
    _,s,_ = svd(λ)
    # need to take the absolute squares of the values
    s_abs = np.square(np.abs(s))
    Sρ = -np.dot(s_abs, np.log2(s_abs))
    return Sρ


In [69]:
entropies = []
for env in envs:
    entropies.append(env_to_entropy(env))

### Finding Parameterization of the 2 parameter scar mps

In [5]:
A = lambda ϕ, θ: np.array([[[0, 1j*np.exp(-1j*ϕ)], 
                            [0,0]],
                          [[np.cos(θ), 0],
                           [np.sin(θ), 0]]])

U = lambda θ, ϕ: np.array([
    [0,1j*np.exp(-1j*ϕ),0,0],
    [np.cos(θ), 0, 0, -np.sin(θ)],
    [0,0,1,0],
    [np.sin(θ), 0, 0, np.cos(θ)]
])

for x,y in zip(np.random.randn(10), np.random.randn(10)):
    assert np.allclose(U(x,y).conj().T@U(x,y), np.eye(4))

In [146]:
class testAnsatz(cirq.Gate):
    def __init__(self, ϕ, θ):
        self.θ = θ
        self.ϕ = ϕ
        
    def num_qubits(self):
        return 2
    
    def _decompose_(self, qubits):
        q = qubits
        return [
            cirq.ZPowGate()
            cirq.ZPowGate(exponent = (self.ϕ - np.pi/2)/np.pi).on(q[1]),
            cirq.X(q[0]),
            cirq.CNOT(control = q[0],target = q[1]),
            cirq.X(q[0]),
            cirq.CNotPowGate(exponent = 2*self.θ/np.pi).on(control = q[1],target = q[0]),
            cirq.S(q[0])
        ]
    
gate = testAnsatz(0, np.pi/2)

ap(cirq.unitary(gate), _abs = False)

[[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]]


#### Exact Hamiltonian for simulation

In [34]:
import numpy as np
from numpy import kron
from functools import reduce
from scipy.linalg import expm
from qmps.represent import Tensor
def multi_tensor(Ops):
    return reduce(kron, Ops)

### Exact hamiltonian simulation for prof of principle ###
P = np.array([[0,0],[0,1]])
X = np.array([[0,1],[1,0]])
n = np.array([[1,0],[0,0]])
I = np.eye(2)

H = lambda μ:0.5*(multi_tensor([I,P,X,P]) + multi_tensor([P,X,P,I])) + (μ/4) * (multi_tensor([I,I,I,n]) + 
                                                                                multi_tensor([I,I,n,I]) +
                                                                                multi_tensor([I,n,I,I]) + 
                                                                                multi_tensor([n,I,I,I]))

W = lambda μ, dt: Tensor(expm(1j * dt * H(μ)),'H')

In [6]:
from qmps.time_evolve_tools import merge

# bUILDING THE TWO SITES FROM As

def two_site_As(θ, ϕ, θ_, ϕ_):
    A1 = A(ϕ, θ)
    A2 = A(ϕ_, θ_)
    return merge(A1,A2)

def generate_two_site_unitary(params):
    return tensor_to_unitary(two_site_As(*params))

##### Using the Direct Parametrization to build the 2-site unitary:

In [67]:
class ScarGate(cirq.Gate):
    def __init__(self, params):
        self.params = params # this order: [θ, ϕ, θ', ϕ']
        
    def num_qubits(self):
        return 3
    
    def _decompose_(self, qubits):
        q = qubits
        π = np.pi
        θ, ϕ, θ_, ϕ_ = self.params
        return [
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ_ + (π/2))).on(q[2]),
            cirq.X.on(q[1]),
            cirq.CNOT(q[1], q[2]),
            cirq.X.on(q[1]),
            cirq.CNotPowGate(exponent=2*θ_/π).on(q[2], q[1]),
            cirq.S.on(q[1]),
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ + (π/2))).on(q[1]),
            cirq.X.on(q[0]),
            cirq.CNOT(q[0], q[1]),
            cirq.X.on(q[0]),
            cirq.CNotPowGate(exponent=2*θ/π).on(q[1], q[0]),
            cirq.S.on(q[0])
        ]
    
u_scar = ScarGate([1,2,3,4])

q = cirq.LineQubit.range(3)
c = cirq.Circuit.from_ops([cirq.decompose_once(u_scar.on(*q))])
print(c)
        

0: ───X───────────────────────────────────────────@───X───X──────────S───
                                                  │       │
1: ───X─────────@───X───X───────────S───Z^0.137───X───────@^(7/11)───────
                │       │
2: ───Z^0.773───X───────@^(-1/11)────────────────────────────────────────


##### Trying to get the environment exactly

In [72]:
import sympy as sp
from sympy import symbols,init_printing, I
init_printing(use_latex='mathjax')

θ1, ϕ1, θ2, ϕ2 = symbols('theta_1 phi_1 theta_2 phi_2', real = True)
θ1, ϕ2

A_sp = lambda θ, ϕ: sp.Array([[[0, I*sp.exp(-I*ϕ)], 
                               [0,0]],
                               [[sp.cos(θ), 0],
                               [sp.sin(θ), 0]]])




A1 = A_sp(θ1, ϕ1)
A2 = A_sp(θ2, ϕ2)

from sympy import tensorproduct, tensorcontraction, permutedims

def merge_sp(A1, A2):
    # Merge two Bond Dim 2 Matrix Product States using Sympy
    contracted = tensorcontraction(tensorproduct(A1,A2),(2,4))
    permuted = permutedims(contracted, (0,2,1,3))
    return permuted.reshape(4,2,2)

A12 = merge_sp(A1,A2)
A12 [2,:,:] 

def transfer_matrix_sp(A12):
    # contract two 'merged' objects on their physical index, and group the remaining indices so it is a 4x4 matrix
    flipped = permutedims(A12, (0,1,2))
    flipped = flipped.conjugate()
    contracted = tensorcontraction(tensorproduct(A12, flipped),(0,3))
    grouped = permutedims(contracted,(0,2,1,3))
    return grouped.reshape(4,4).tomatrix()

tm = transfer_matrix_sp(A12)

result = tm.eigenvects()
environment = result[1][2][0].reshape(2,2)
environment

⎡                                        2                                    
⎢                                    -cos (θ₁)                                
⎢                         ────────────────────────────────                    
⎢                            2          2        2                            
⎢                         sin (θ₂) + cos (θ₁)⋅cos (θ₂) - 1                    
⎢                                                                             
⎢ ⎛⎛   2          2        2        ⎞                              3        2 
⎢-⎝⎝sin (θ₂) + cos (θ₁)⋅cos (θ₂) - 1⎠⋅sin(θ₁)⋅cos(θ₁) - sin(θ₁)⋅cos (θ₁)⋅cos (
⎢─────────────────────────────────────────────────────────────────────────────
⎢                             2          2        2                           
⎣                        - sin (θ₂) - cos (θ₁)⋅cos (θ₂) + 1                   

        ⎛⎛   2          2        2        ⎞                              3    
       -⎝⎝sin (θ₂) + cos (θ₁)⋅cos (θ₂) - 1⎠⋅sin(θ₁)

The analytical expression is not that useful since it appears too complicated to easily express with rotation gates

##### Making the Hamiltonian:

In [99]:
# How to build custom controlled gates:
# - IMPORTANT: LATER VERSIONS OF CIRQ HAVE CHANGED CONTROLLED GATE AND THIS WILL HAVE TO BE CHANGE IF WE UPDATE

q = cirq.LineQubit.range(3)
c = cirq.Circuit.from_ops(
    cirq.ControlledGate(
        sub_gate=cirq.XPowGate(),
        num_controls = 2, 
        control_qubits=[q[0], q[2]]).on(q[1])   
)


In [4]:
class PXPHamiltonian(cirq.Gate):
    def __init__(self, dt):
        self.dt = dt
    
    def num_qubits(self):
        return 4
    
    def _decompose_(self, qubits):
        q = qubits
        return[
            cirq.X.on(q[0]), cirq.X.on(q[2]),
            
            cirq.ControlledGate(
            sub_gate = cirq.XPowGate(exponent=self.dt / np.pi),
            num_controls = 2,
            control_qubits = [q[0], q[2]]).on(q[1]),
            
            cirq.X.on(q[0]), cirq.X.on(q[2]),
            
            cirq.X.on(q[1]), cirq.X.on(q[3]),

            cirq.ControlledGate(
            sub_gate = cirq.XPowGate(exponent=self.dt / np.pi),
            num_controls = 2,
            control_qubits = [q[1], q[3]]).on(q[2]),
            
            cirq.X.on(q[1]), cirq.X.on(q[3])

        ]
    
    def _circuit_diagram_info_(self, args):
        return ['H']*self.num_qubits()

    




0: ───X───@─────────X─────────────────
          │
1: ───────X─────────X───@─────────X───
          │             │
2: ───X───@^0.032───X───X─────────────
                        │
3: ───X─────────────────@^0.032───X───


In [18]:
from xmps.iMPS import Map
from qmps.time_evolve_tools import put_env_on_right_site, put_env_on_left_site
from qmps.represent import Environment 
class ScarGate(cirq.Gate):
    def __init__(self, params):
        self.params = params # this order: [θ, ϕ, θ', ϕ']

    def num_qubits(self):
        return 3

    def _decompose_(self, qubits):
        q = qubits
        π = np.pi
        θ, ϕ, θ_, ϕ_ = self.params
        return [
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ_ + (π/2))).on(q[2]),
            cirq.X.on(q[1]),
            cirq.CNOT(q[1], q[2]),
            cirq.X.on(q[1]),
            cirq.CNotPowGate(exponent=2*θ_/π).on(q[2], q[1]),
            cirq.S.on(q[1]),
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ + (π/2))).on(q[1]),
            cirq.X.on(q[0]),
            cirq.CNOT(q[0], q[1]),
            cirq.X.on(q[0]),
            cirq.CNotPowGate(exponent=2*θ/π).on(q[1], q[0]),
            cirq.S.on(q[0])
        ]
    
    def _circuit_diagram_info_(self, args):
        return ['U']*self.num_qubits()

A = lambda θ, ϕ: np.array([[[0, 1j*np.exp(-1j*ϕ)], 
                            [0,0]],
                           [[np.cos(θ), 0],
                            [np.sin(θ), 0]]])


ham = PXPHamiltonian(0.1)

def scars_time_evolve_cost_function(params, current_params, ham):
    '''
    params are formatted like: [θ1, ϕ1, θ2, ϕ2]
    '''    
    θ1, ϕ1, θ2, ϕ2 = current_params
    θ1_,ϕ1_,θ2_, ϕ2_ = params

    A1 = A(θ1, ϕ1)
    A2 = A(θ2, ϕ2)
    A1_= A(θ1_, ϕ1_)
    A2_= A(θ2_, ϕ2_)
    
    _, r = Map(merge(A1,A2), merge(A1_,A2_)).right_fixed_point()
    R = Environment(put_env_on_left_site(r), 'R')
    L = Environment(put_env_on_right_site(r.conj().T),'L')
    
    U12 = ScarGate(current_params)
    U12_= ScarGate(params)
    q = cirq.LineQubit.range(8)
    circuit = cirq.Circuit.from_ops([
        cirq.H(q[5]),
        cirq.CNOT(q[5],q[6]),
        U12(*q[3:6]),
        U12(*q[1:4]),
        L(*q[0:2]),
        ham(*q[2:6]),
        R(*q[6:8]),
        cirq.inverse(U12_(*q[1:4])),
        cirq.inverse(U12_(*q[3:6])),
        cirq.CNOT(q[5],q[6]),
        cirq.H(q[5])
    ])
    
    # print(circuit.to_text_diagram(transpose = True))
    sim = cirq.Simulator()
    ψ = sim.simulate(circuit).final_state[0]
    return -np.abs(ψ)*2

scars_time_evolve_cost_function(current_params, current_params, hamiltonian)

-0.9999904036521912

In [22]:
from scipy.optimize import minimize
from tqdm.notebook import tqdm
timesteps = 2000
hamiltonian = PXPHamiltonian(0.01)
current_params = [0,np.random.randn(1)[0], np.pi/2, np.random.randn(1)[0]]
final_params = []

for _ in tqdm(range(timesteps)):
    final_params.append(current_params)
    initial_params = current_params
    res = minimize(scars_time_evolve_cost_function, initial_params, args = (current_params, hamiltonian), options = {'disp':True}, method = 'Nelder-Mead')
    current_params = res.x

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 33
         Function evaluations: 92
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 38
         Function evaluations: 103
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 31
         Function evaluations: 91
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 28
         Function evaluations: 95
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 36
         Function evaluations: 95
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 39
         Function evaluations: 109
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 28
         Function evaluations: 90
Optimization terminated s

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 27
         Function evaluations: 84
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 27
         Function evaluations: 92
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 28
         Function evaluations: 98
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 38
         Function evaluations: 102
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 42
         Function evaluations: 113
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 28
         Function evaluations: 88
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 34
         Function evaluations: 98
Optimization terminated s

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 32
         Function evaluations: 104
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 37
         Function evaluations: 104
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 43
         Function evaluations: 112
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 38
         Function evaluations: 105
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 32
         Function evaluations: 100
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 43
         Function evaluations: 113
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 43
         Function evaluations: 113
Optimization termina

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 32
         Function evaluations: 100
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 48
         Function evaluations: 127
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 34
         Function evaluations: 99
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 37
         Function evaluations: 111
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 23
         Function evaluations: 84
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 37
         Function evaluations: 107
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 29
         Function evaluations: 92
Optimization terminated

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 44
         Function evaluations: 110
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 30
         Function evaluations: 103
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 23
         Function evaluations: 80
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 33
         Function evaluations: 98
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 36
         Function evaluations: 99
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 38
         Function evaluations: 98
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 35
         Function evaluations: 101
Optimization terminated 

...

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 75
         Function evaluations: 230
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 69
         Function evaluations: 217
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 83
         Function evaluations: 233
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 76
         Function evaluations: 222
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 83
         Function evaluations: 238
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 80
         Function evaluations: 229
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 71
         Function evaluations: 234
Optimization termina

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 77
         Function evaluations: 228
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 81
         Function evaluations: 240
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 83
         Function evaluations: 246
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 73
         Function evaluations: 234
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 73
         Function evaluations: 252
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 77
         Function evaluations: 237
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 84
         Function evaluations: 240
Optimization termina

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 75
         Function evaluations: 232
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 84
         Function evaluations: 241
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 75
         Function evaluations: 239
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 72
         Function evaluations: 230
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 75
         Function evaluations: 234
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 84
         Function evaluations: 250
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 76
         Function evaluations: 240
Optimization termina

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 83
         Function evaluations: 245
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 89
         Function evaluations: 258
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 80
         Function evaluations: 243
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 86
         Function evaluations: 258
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 85
         Function evaluations: 266
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 86
         Function evaluations: 248
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 84
         Function evaluations: 256
Optimization termina

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 77
         Function evaluations: 246
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 87
         Function evaluations: 266
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 70
         Function evaluations: 251
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 77
         Function evaluations: 249
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 82
         Function evaluations: 249
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 92
         Function evaluations: 263
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 88
         Function evaluations: 262
Optimization termina

## Putting it all together:

In [ ]:
#### Use this for a shallow representation of the PXP, 
#### otherwise use the W lambda function which is exact but might be really deep on chip ###

class PXPHamiltonian(cirq.Gate):
    def __init__(self, dt):
        self.dt = dt
    
    def num_qubits(self):
        return 4
    
    def _decompose_(self, qubits):
        q = qubits
        return[
            cirq.X.on(q[0]), cirq.X.on(q[2]),
            
            cirq.ControlledGate(
            sub_gate = cirq.XPowGate(exponent=self.dt / np.pi),
            num_controls = 2,
            control_qubits = [q[0], q[2]]).on(q[1]),
            
            cirq.X.on(q[0]), cirq.X.on(q[2]),
            
            cirq.X.on(q[1]), cirq.X.on(q[3]),

            cirq.ControlledGate(
            sub_gate = cirq.XPowGate(exponent=self.dt / np.pi),
            num_controls = 2,
            control_qubits = [q[1], q[3]]).on(q[2]),
            
            cirq.X.on(q[1]), cirq.X.on(q[3])

        ]
    
    def _circuit_diagram_info_(self, args):
        return ['H']*self.num_qubits()


In [10]:
import numpy as np
from numpy import kron
from functools import reduce
from scipy.linalg import expm
from qmps.represent import Tensor, Environment
import cirq
from qmps.time_evolve_tools import merge, put_env_on_left_site, put_env_on_right_site
from xmps.iMPS import Map
from scipy.optimize import minimize
from tqdm.notebook import tqdm 

def multi_tensor(Ops):
    return reduce(kron, Ops)

### Exact hamiltonian simulation for prof of principle ###
P = np.array([[0,0],[0,1]])
X = np.array([[0,1],[1,0]])
n = np.array([[1,0],[0,0]])
I = np.eye(2)

H = lambda μ:0.5*(multi_tensor([I,P,X,P]) + multi_tensor([P,X,P,I])) + (μ/4) * (multi_tensor([I,I,I,n]) + 
                                                                                multi_tensor([I,I,n,I]) +
                                                                                multi_tensor([I,n,I,I]) + 
                                                                                multi_tensor([n,I,I,I]))

W = lambda μ, dt: Tensor(expm(1j * dt * H(μ)),'H')

class ScarGate(cirq.Gate):
    def __init__(self, params):
        self.params = params # this order: [θ, ϕ, θ', ϕ']

    def num_qubits(self):
        return 3

    def _decompose_(self, qubits):
        q = qubits
        π = np.pi
        θ, ϕ, θ_, ϕ_ = self.params
        return [
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ_ + (π/2))).on(q[2]),
            cirq.X.on(q[1]),
            cirq.CNOT(q[1], q[2]),
            cirq.X.on(q[1]),
            cirq.CNotPowGate(exponent=2*θ_/π).on(q[2], q[1]),
            cirq.S.on(q[1]),
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ + (π/2))).on(q[1]),
            cirq.X.on(q[0]),
            cirq.CNOT(q[0], q[1]),
            cirq.X.on(q[0]),
            cirq.CNotPowGate(exponent=2*θ/π).on(q[1], q[0]),
            cirq.S.on(q[0])
        ]
    
    def _circuit_diagram_info_(self, args):
        return ['U']*self.num_qubits()

A = lambda θ, ϕ: np.array([[[0, 1j*np.exp(-1j*ϕ)], 
                            [0,0]],
                           [[np.cos(θ), 0],
                            [np.sin(θ), 0]]])


def scars_time_evolve_cost_function(params, current_params, ham):
    '''
    params are formatted like: [θ1, ϕ1, θ2, ϕ2]
    '''    
    θ1, ϕ1, θ2, ϕ2 = current_params
    θ1_,ϕ1_,θ2_, ϕ2_ = params

    A1 = A(θ1, ϕ1)
    A2 = A(θ2, ϕ2)
    A1_= A(θ1_, ϕ1_)
    A2_= A(θ2_, ϕ2_)
    
    _, r = Map(merge(A1,A2), merge(A1_,A2_)).right_fixed_point()
    R = Environment(put_env_on_left_site(r), 'R')
    L = Environment(put_env_on_right_site(r.conj().T),'L')
    
    U12 = ScarGate(current_params)
    U12_= ScarGate(params)
    q = cirq.LineQubit.range(8)
    circuit = cirq.Circuit.from_ops([
        cirq.H(q[5]),
        cirq.CNOT(q[5],q[6]),
        U12(*q[3:6]),
        U12(*q[1:4]),
        L(*q[0:2]),
        ham(*q[2:6]),
        R(*q[6:8]),
        cirq.inverse(U12_(*q[1:4])),
        cirq.inverse(U12_(*q[3:6])),
        cirq.CNOT(q[5],q[6]),
        cirq.H(q[5])
    ])
    
    # print(circuit.to_text_diagram(transpose = True))
    sim = cirq.Simulator()
    ψ = sim.simulate(circuit).final_state[0]
    return -np.abs(ψ)*2


def simulate_scars(dt, timesteps, μ, initial_params, save_file = None):
    hamiltonian = W(μ, dt)
    final_params = []
    current_params = initial_params
    for _ in tqdm(range(timesteps)):
        final_params.append(np.mod(current_params, 2*np.pi))
        res = minimize(scars_time_evolve_cost_function, current_params, args = (current_params, hamiltonian), options = {'disp':True}, method = 'Nelder-Mead')
        current_params = res.x
    
    if save_file:
        np.save(save_file, np.array(final_params))
    
    return final_params
    

In [8]:
(500 / 0.05 * 2) / 3600

5.555555555555555

In [12]:
final_params = simulate_scars(0.01, 1000, 0.325, [np.pi/2,0,0,0], save_file='run_2_1000_0_01' )

Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 21
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 29
         Function evaluations: 67
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 21
         Function evaluations: 55
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 21
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 21
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 22
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 22
         Function evaluations: 54
Optimization terminated suc

Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 29
         Function evaluations: 68
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 25
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 27
         Function evaluations: 65
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 26
         Function evaluations: 57
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 23
         Function evaluations: 54
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 21
         Function evaluations: 49
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 26
         Function evaluations: 57
Optimization terminated suc

Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 20
         Function evaluations: 53
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 22
         Function evaluations: 52
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 21
         Function evaluations: 50
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 19
         Function evaluations: 52
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 26
         Function evaluations: 58
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 24
         Function evaluations: 60
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 23
         Function evaluations: 56
Optimization terminated suc

Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 22
         Function evaluations: 52
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 24
         Function evaluations: 58
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 22
         Function evaluations: 53
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 24
         Function evaluations: 50
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 23
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 19
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 24
         Function evaluations: 58
Optimization terminated suc

Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 20
         Function evaluations: 48
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 25
         Function evaluations: 57
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 24
         Function evaluations: 57
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 26
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 20
         Function evaluations: 47
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 24
         Function evaluations: 59
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 23
         Function evaluations: 55
Optimization terminated suc

...

Optimization terminated successfully.
         Current function value: -0.999985
         Iterations: 29
         Function evaluations: 69
Optimization terminated successfully.
         Current function value: -0.999986
         Iterations: 31
         Function evaluations: 71
Optimization terminated successfully.
         Current function value: -0.999986
         Iterations: 32
         Function evaluations: 73
Optimization terminated successfully.
         Current function value: -0.999986
         Iterations: 32
         Function evaluations: 76
Optimization terminated successfully.
         Current function value: -0.999985
         Iterations: 34
         Function evaluations: 74
Optimization terminated successfully.
         Current function value: -0.999985
         Iterations: 29
         Function evaluations: 64
Optimization terminated successfully.
         Current function value: -0.999985
         Iterations: 29
         Function evaluations: 69
Optimization terminated suc

Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 30
         Function evaluations: 71
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 34
         Function evaluations: 75
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 28
         Function evaluations: 67
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 29
         Function evaluations: 73
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 34
         Function evaluations: 76
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 31
         Function evaluations: 72
Optimization terminated successfully.
         Current function value: -0.999988
         Iterations: 31
         Function evaluations: 73
Optimization terminated suc

Optimization terminated successfully.
         Current function value: -0.999990
         Iterations: 31
         Function evaluations: 71
Optimization terminated successfully.
         Current function value: -0.999990
         Iterations: 26
         Function evaluations: 64
Optimization terminated successfully.
         Current function value: -0.999990
         Iterations: 29
         Function evaluations: 66
Optimization terminated successfully.
         Current function value: -0.999990
         Iterations: 30
         Function evaluations: 71
Optimization terminated successfully.
         Current function value: -0.999990
         Iterations: 30
         Function evaluations: 70
Optimization terminated successfully.
         Current function value: -0.999990
         Iterations: 29
         Function evaluations: 68
Optimization terminated successfully.
         Current function value: -0.999990
         Iterations: 27
         Function evaluations: 65
Optimization terminated suc

Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 33
         Function evaluations: 73
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 33
         Function evaluations: 72
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 35
         Function evaluations: 77
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 30
         Function evaluations: 68
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 35
         Function evaluations: 79
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 34
         Function evaluations: 72
Optimization terminated successfully.
         Current function value: -0.999992
         Iterations: 35
         Function evaluations: 79
Optimization terminated suc

Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 32
         Function evaluations: 80
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 36
         Function evaluations: 86
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 32
         Function evaluations: 72
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 31
         Function evaluations: 75
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 32
         Function evaluations: 75
Optimization terminated successfully.
         Current function value: -0.999994
         Iterations: 36
         Function evaluations: 82
Optimization terminated successfully.
         Current function value: -0.999993
         Iterations: 31
         Function evaluations: 80
Optimization terminated suc

#### ODE Analytical Solver:

In [64]:
from numpy import sin, cos, tan, arcsin, pi
import numpy as np
from scipy.integrate import odeint
dθdt = lambda θ1, ϕ1, ϕ2, θ2: tan(θ2)*sin(θ1)*(cos(θ1)**2)*cos(ϕ1) + cos(θ2)*cos(ϕ2)
dϕdt = lambda θ1, ϕ1, ϕ2, θ2: 2*tan(θ1)*cos(θ2)*sin(ϕ2) - 0.5*tan(θ2)*cos(θ1)*sin(ϕ1)*(2*(sin(θ2)**-2) + 
                                                                                                cos(2*θ1) - 5)

def func_list(angles,t,μ):
    return[dθdt(*angles), -μ + dϕdt(*angles), -μ + dϕdt(*reversed(angles)), dθdt(*reversed(angles))]

np.random.seed(0)

y0 = np.random.randn(4)
steps = 500
t = np.linspace(0.0, 100.0, steps)
dt = t[1]-t[0]
μ = 0.325

classical_angles = odeint(func_list, y0, t, args=(μ,))
quantum_angles = simulate_scars(dt, steps, μ, y0, save_file=f'{steps}_{dt}_{μ}_random_initial')


Optimization terminated successfully.
         Current function value: -0.996533
         Iterations: 87
         Function evaluations: 161
Optimization terminated successfully.
         Current function value: -0.996354
         Iterations: 84
         Function evaluations: 162
Optimization terminated successfully.
         Current function value: -0.996210
         Iterations: 68
         Function evaluations: 130
Optimization terminated successfully.
         Current function value: -0.996107
         Iterations: 97
         Function evaluations: 193
Optimization terminated successfully.
         Current function value: -0.996053
         Iterations: 83
         Function evaluations: 166
Optimization terminated successfully.
         Current function value: -0.996041
         Iterations: 80
         Function evaluations: 160
Optimization terminated successfully.
         Current function value: -0.996063
         Iterations: 73
         Function evaluations: 157
Optimization termina

Optimization terminated successfully.
         Current function value: -0.995707
         Iterations: 91
         Function evaluations: 177
Optimization terminated successfully.
         Current function value: -0.995643
         Iterations: 83
         Function evaluations: 159
Optimization terminated successfully.
         Current function value: -0.995580
         Iterations: 78
         Function evaluations: 150
Optimization terminated successfully.
         Current function value: -0.995526
         Iterations: 70
         Function evaluations: 138
Optimization terminated successfully.
         Current function value: -0.995482
         Iterations: 60
         Function evaluations: 126
Optimization terminated successfully.
         Current function value: -0.995453
         Iterations: 67
         Function evaluations: 134
Optimization terminated successfully.
         Current function value: -0.995440
         Iterations: 82
         Function evaluations: 157
Optimization termina

Optimization terminated successfully.
         Current function value: -0.996187
         Iterations: 73
         Function evaluations: 146
Optimization terminated successfully.
         Current function value: -0.996037
         Iterations: 72
         Function evaluations: 143
Optimization terminated successfully.
         Current function value: -0.995859
         Iterations: 70
         Function evaluations: 149
Optimization terminated successfully.
         Current function value: -0.995688
         Iterations: 73
         Function evaluations: 159
Optimization terminated successfully.
         Current function value: -0.995564
         Iterations: 75
         Function evaluations: 151
Optimization terminated successfully.
         Current function value: -0.995485
         Iterations: 86
         Function evaluations: 176
Optimization terminated successfully.
         Current function value: -0.995444
         Iterations: 99
         Function evaluations: 192
Optimization termina

Optimization terminated successfully.
         Current function value: -0.995566
         Iterations: 55
         Function evaluations: 117
Optimization terminated successfully.
         Current function value: -0.995557
         Iterations: 58
         Function evaluations: 118
Optimization terminated successfully.
         Current function value: -0.995548
         Iterations: 53
         Function evaluations: 104
Optimization terminated successfully.
         Current function value: -0.995539
         Iterations: 52
         Function evaluations: 109
Optimization terminated successfully.
         Current function value: -0.995529
         Iterations: 52
         Function evaluations: 108
Optimization terminated successfully.
         Current function value: -0.995519
         Iterations: 60
         Function evaluations: 121
Optimization terminated successfully.
         Current function value: -0.995508
         Iterations: 64
         Function evaluations: 127
Optimization termina

Optimization terminated successfully.
         Current function value: -0.995359
         Iterations: 56
         Function evaluations: 117
Optimization terminated successfully.
         Current function value: -0.995368
         Iterations: 68
         Function evaluations: 134
Optimization terminated successfully.
         Current function value: -0.995378
         Iterations: 64
         Function evaluations: 130
Optimization terminated successfully.
         Current function value: -0.995387
         Iterations: 82
         Function evaluations: 157
Optimization terminated successfully.
         Current function value: -0.995397
         Iterations: 94
         Function evaluations: 177
Optimization terminated successfully.
         Current function value: -0.995406
         Iterations: 83
         Function evaluations: 156
Optimization terminated successfully.
         Current function value: -0.995416
         Iterations: 77
         Function evaluations: 147
Optimization termina

Optimization terminated successfully.
         Current function value: -0.995234
         Iterations: 57
         Function evaluations: 123
Optimization terminated successfully.
         Current function value: -0.995247
         Iterations: 53
         Function evaluations: 116
Optimization terminated successfully.
         Current function value: -0.995261
         Iterations: 58
         Function evaluations: 121
Optimization terminated successfully.
         Current function value: -0.995274
         Iterations: 48
         Function evaluations: 113
Optimization terminated successfully.
         Current function value: -0.995288
         Iterations: 55
         Function evaluations: 122
Optimization terminated successfully.
         Current function value: -0.995304
         Iterations: 58
         Function evaluations: 127
Optimization terminated successfully.
         Current function value: -0.995319
         Iterations: 58
         Function evaluations: 116
Optimization termina

Optimization terminated successfully.
         Current function value: -0.995799
         Iterations: 58
         Function evaluations: 121
Optimization terminated successfully.
         Current function value: -0.995781
         Iterations: 53
         Function evaluations: 121
Optimization terminated successfully.
         Current function value: -0.995763
         Iterations: 64
         Function evaluations: 133
Optimization terminated successfully.
         Current function value: -0.995744
         Iterations: 61
         Function evaluations: 126
Optimization terminated successfully.
         Current function value: -0.995725
         Iterations: 57
         Function evaluations: 123
Optimization terminated successfully.
         Current function value: -0.995705
         Iterations: 55
         Function evaluations: 117
Optimization terminated successfully.
         Current function value: -0.995685
         Iterations: 74
         Function evaluations: 154
Optimization termina

Optimization terminated successfully.
         Current function value: -0.995722
         Iterations: 60
         Function evaluations: 130
Optimization terminated successfully.
         Current function value: -0.995725
         Iterations: 58
         Function evaluations: 123
Optimization terminated successfully.
         Current function value: -0.995727
         Iterations: 62
         Function evaluations: 130
Optimization terminated successfully.
         Current function value: -0.995728
         Iterations: 60
         Function evaluations: 120
Optimization terminated successfully.
         Current function value: -0.995727
         Iterations: 58
         Function evaluations: 125
Optimization terminated successfully.
         Current function value: -0.995727
         Iterations: 61
         Function evaluations: 125
Optimization terminated successfully.
         Current function value: -0.995725
         Iterations: 57
         Function evaluations: 118
Optimization termina

Optimization terminated successfully.
         Current function value: -0.995493
         Iterations: 53
         Function evaluations: 119
Optimization terminated successfully.
         Current function value: -0.995496
         Iterations: 52
         Function evaluations: 114
Optimization terminated successfully.
         Current function value: -0.995498
         Iterations: 56
         Function evaluations: 126
Optimization terminated successfully.
         Current function value: -0.995501
         Iterations: 50
         Function evaluations: 111
Optimization terminated successfully.
         Current function value: -0.995503
         Iterations: 61
         Function evaluations: 124
Optimization terminated successfully.
         Current function value: -0.995504
         Iterations: 64
         Function evaluations: 134
Optimization terminated successfully.
         Current function value: -0.995506
         Iterations: 54
         Function evaluations: 116
Optimization termina

In [67]:
θ1q = np.array(quantum_angles)[1:]

(500, 4)

In [48]:
θ1

array([0.19696306, 0.29427369, 0.3948233 , ..., 0.88571402, 0.80155187,
       0.70201476])